In [52]:
import numpy as np
import pandas as pd
import statsmodels.stats.api as sms

Продукт – подписка на репетиторов по ЕГЭ. Аудитория может приобрести пакет уроков по разным предметам. Учеба проходит на онлайн-платформе по программам обучения с уклоном в конкретную область: можно решать задачки, смотреть лекции и заниматься с репетитором.

In [40]:
df = pd.read_csv('hw1.csv', encoding='utf-8', delimiter=';', index_col=0)
df.head()

,program_name,performance,is_ege,age
1,math&phys,"0,60620346525684",no,17
2,math&phys,"0,648849559854716",yes,18
3,math&phys,"0,729141345340759",no,17
4,math&phys,"0,863283115997911",yes,17
5,math&phys,"0,580672772414982",no,17


In [41]:
df['performance'] = pd.to_numeric([i.replace(',','.') for i in df['performance'].values])

In [27]:
df['program_name'].value_counts(normalize=True)

russ&lit     0.714286
math&phys    0.285714
Name: program_name, dtype: float64

Посмотрим, входят ли группы программ в один доверительный интервал по успеваемости.

In [51]:
df.groupby('program_name')['performance'].describe()

,count,mean,std,min,25%,50%,75%,max
program_name,,,,,,,,
math&phys,10.0,0.720606,0.126262,0.524715,0.616865,0.740393,0.835597,0.877870
russ&lit,25.0,0.797763,0.110245,0.605356,0.706888,0.792832,0.887047,0.996762


In [77]:
sms.DescrStatsW(df.loc[df['program_name'] == 'math&phys', 'performance'], ddof=1).tconfint_mean()

(0.6302830568697645, 0.8109280397373847)

In [64]:
sms.DescrStatsW(df.loc[df['program_name'] == 'russ&lit', 'performance'], ddof=1).tconfint_mean()

(0.7522563835989913, 0.8432703759062805)

Доверительные интервалы для среднего получились довольно широкими, причём для math&phys границы шире. Посмотрим на дисперсию.

In [94]:
sms.DescrStatsW(df.loc[df['program_name'] == 'russ&lit', 'performance']).var_ddof(1)

0.012154011714718157

In [95]:
sms.DescrStatsW(df.loc[df['program_name'] == 'math&phys', 'performance']).var_ddof(1)

0.01594212715880291

Дисперсия в группах отличается всего примерно на 0.003 п.п. - различие крайне незначительное. Поэтому можно утверждать, что
широкие границы интервалов обусловлены малым размером выборок. Построим доверительные интервалы для дисперсии.

In [116]:
#таблица с распределением хи квадрат - альфа пополам и 1-альфа пополам для 95%
print(f"Левая граница для math&phys {(len(df[df['program_name'] == 'math&phys']) - 1) * np.var(df.loc[df['program_name'] == 'math&phys', 'performance'], ddof=1) / 19.02}")
print(f"Правая граница для math&phys {(len(df[df['program_name'] == 'math&phys']) - 1) * np.var(df.loc[df['program_name'] == 'math&phys', 'performance'], ddof=1) / 2.7}")

Левая граница для math&phys 0.007543593292808949
Правая граница для math&phys 0.053140423862676374


In [117]:
print(f"Левая граница для russ&lit {(len(df[df['program_name'] == 'russ&lit']) - 1) * np.var(df.loc[df['program_name'] == 'russ&lit', 'performance'], ddof=1) / 39.36}")
print(f"Правая граница для russ&lit {(len(df[df['program_name'] == 'russ&lit']) - 1) * np.var(df.loc[df['program_name'] == 'russ&lit', 'performance'], ddof=1) / 12.4}")

Левая граница для russ&lit 0.007410982752876925
Правая граница для russ&lit 0.02352389364138998


Уже чуть получше, но можно и лучше. Границы доверительных интервалов диперсий пересекаются, причём границы для ruslit входят в
mathphys(опять влияние развмера). С вероятностью 95 процентов можем утверждать, что по результатам успеваемости две программы одинаковые. Однако, нужно иметь в виду риск ошибки в результате малого объёма выборки(наблюдений).

Посмотрим у какой группы is_ege разброс в успеваемости больше.

In [75]:
df.groupby('is_ege')['performance'].describe()

,count,mean,std,min,25%,50%,75%,max
is_ege,,,,,,,,
no,25.0,0.768338,0.128912,0.524715,0.674487,0.752014,0.874809,0.996762
yes,10.0,0.794170,0.090735,0.648850,0.753843,0.780868,0.862301,0.930949


Средние значения, стандартное отклонение и медиана отличаются незначительно, а вот минимумы и максимумы довольно сильно отличаются. Для группы yes межквартильный размах существенно уже, а стандартное отклонение меньше.

In [90]:
sms.DescrStatsW(df.loc[df['is_ege'] == 'yes', 'performance']).var_ddof(1)

0.008232850516950553

In [91]:
sms.DescrStatsW(df.loc[df['is_ege'] == 'no', 'performance']).var_ddof(1)

0.016618214678310043

Диперсия же между группами практически не различается - отличие примерно всего на 0.008 п.п. Поэтому можно утверждать, что
статистические различия обусловлены малым размером выборки(наблюдений) для yes.

**Гипотезы и выводы**
 - малый объём выборки: есть риск получить ошибку в тестах на распределения и сделать некорректные выводы о генеральной совокупности
 - малый объём наблюдений по группам program_name и is_age: аналогичный риск, описанный выше + риск получить некорректные результаты при сравнении групп. Пока можно говорить о том, что с вероятностью 95% различий в программах по успеваемости и различий в успеваемости между сдающими и несдающими ЕГЭ в этом году не наблюдается.
 - необходимо определить достаточный(более комфортный) объём выборки и наблюдений по группам и/или хорошо поработать со
 сплитом.